In [1]:
from preprocess import (
    preprocess_monthly_file, 
    preprocessed_filename,
    preprocess_region,
    read_region,
    read_preprocessed,
)

In [2]:
t = (2022, 8, 19, "AM")
# region = ('CONUS', [-126.401171875,24.071240929282325,
#                     -61.36210937500001,49.40003415463647])
region = ('WesternUS',[-125.698046875,31.676476158707615,
                       -101.00078125,49.51429477264348])
sat = "SNPP"

## Once per region
Start by preprocessing the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.


In [3]:
preprocess_region(region)

## Once per region per input file

Next process each monthly file for the region.

In [4]:
region = read_region(region)

In [5]:
preprocess_monthly_file(t, region, sat)

preprocessing monthly file for 2022-8, WesternUS, SNPP
filtering...
setting ap and pm...
clustering...
writing processed/SNPP/WesternUS/20220801_AM.txt
clustering...
writing processed/SNPP/WesternUS/20220801_PM.txt
clustering...
writing processed/SNPP/WesternUS/20220802_AM.txt
clustering...
writing processed/SNPP/WesternUS/20220802_PM.txt
clustering...
writing processed/SNPP/WesternUS/20220803_AM.txt
clustering...
writing processed/SNPP/WesternUS/20220803_PM.txt
clustering...
writing processed/SNPP/WesternUS/20220804_AM.txt
clustering...
writing processed/SNPP/WesternUS/20220804_PM.txt
clustering...
writing processed/SNPP/WesternUS/20220805_AM.txt
clustering...
writing processed/SNPP/WesternUS/20220805_PM.txt
clustering...
writing processed/SNPP/WesternUS/20220806_AM.txt
clustering...
writing processed/SNPP/WesternUS/20220806_PM.txt
clustering...
writing processed/SNPP/WesternUS/20220807_AM.txt
clustering...
writing processed/SNPP/WesternUS/20220807_PM.txt
clustering...
writing process

## Run the rest of the algorithm

Now you have pre-clustered data at the half day for the region.

In [ ]:
read_preprocessed(t, region, sat)

## Experiments with hulls

Trying out next steps around hulls:

In [ ]:
df = read_preprocessed(t, region, sat)
df.initial_cid.value_counts()

In [ ]:
%%timeit
for ic, subset in df.groupby("initial_cid"):
    pixels = [
        FirePixel(
            row.x,
            row.y,
            row.Lon,
            row.Lat,
            row.FRP,
            row.DS,
            row.DT,
            row.ampm,
            row.YYYYMMDD_HHMM,
            row.Sat,
            row.origin,
        )
        for row in subset.itertuples()
    ]
    cluster = Cluster(ic, pixels, t, sensor=firessr)
    hull = cluster.hull

In [ ]:
%%timeit
for ic in df.initial_cid.unique():
    pixels = [
        FirePixel(
            row.x,
            row.y,
            row.Lon,
            row.Lat,
            row.FRP,
            row.DS,
            row.DT,
            row.ampm,
            row.YYYYMMDD_HHMM,
            row.Sat,
            row.origin,
        )
        for row in df.loc[df.initial_cid == ic].itertuples()
    ]
    cluster = Cluster(ic, pixels, t, sensor=firessr)
    hull = cluster.hull